In [4]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.utils import to_categorical


In [7]:
def load_data(input_folder, patch_size):
    data = []
    labels = []

    categories = os.listdir(input_folder)
    label_encoder = LabelEncoder()

    for category_idx, category in enumerate(categories):
        category_path = os.path.join(input_folder, category)
        image_files = [f for f in os.listdir(category_path) if f.endswith('.png')]

        for image_file in image_files:
            image_path = os.path.join(category_path, image_file)
            image = cv2.imread(image_path)

            if image is not None and image.size != 0:  # Check if the image is valid
                image = cv2.resize(image, (patch_size, patch_size))  # Resize the image to patch size
                data.append(image)
                labels.append(category_idx)

    data = np.array(data)
    labels = np.array(labels)
    labels = to_categorical(label_encoder.fit_transform(labels))

    return data, labels


def create_cnn_model(input_shape, num_classes):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [6]:
if __name__ == "__main__":
    input_folder = "patched_images"
    patch_size = 64

    # Load data
    data, labels = load_data(input_folder, patch_size)

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

    # Define input shape and number of classes
    input_shape = (patch_size, patch_size, 3)
    num_classes = len(os.listdir("UCMerced_LandUse/Images"))

    # Create and compile the CNN model
    model = create_cnn_model(input_shape, num_classes)

    # Train the model
    model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

    # Evaluate the model
    loss, accuracy = model.evaluate(X_test, y_test)
    print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'
